In [1]:
import os
from dotenv import load_dotenv

import bs4
import langchain
import langchain_community
import langchain_core
import langchain_openai
import lancedb

U:\OpenAI\RAG_openai\RAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.lancedb import LanceDB
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import lancedb

In [3]:
vectordb = lancedb.connect("/temp/lancedb")
vector_table = vectordb.create_table("my_table",
                                    data = [{
            "vector": OpenAIEmbeddings().embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }], mode="overwrite")

In [5]:
bs_strainer = bs4.SoupStrainer(class_ = ("post-content", "post-title", "post-header"))  
loader = WebBaseLoader(web_path="https://lilianweng.github.io/posts/2023-06-23-agent/",
                       bs_kwargs=dict(parse_only = bs_strainer)
                      )
docs = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap = 250)
splits = text_splitter.split_documents(docs)
vectorStore = LanceDB.from_documents(documents=splits, embedding=OpenAIEmbeddings(), connection = vector_table)

In [8]:
retriever = vectorStore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(temperature=0.2)

In [13]:
def format_docs(docs):
    return "/n/n".join(doc.page_content for doc in docs)

In [19]:
rag_chain = (
    RunnablePassthrough.assign(context = (lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
rag_chain_with_resource = RunnableParallel(
    {"context": retriever, "question" : RunnablePassthrough()}
).assign(answer = rag_chain)

In [23]:
res = rag_chain_with_resource.invoke("What is Task Decomposition?")

In [24]:
res

{'context': [Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', metadata={'vector': array([-0.0051782 ,  0.00363057,  0.01988922, ..., -0.00395657,
         -0.01623806, -0.02603854], dtype=float32), 'id': '0b7a54c8-24c8-4b71-8574-f4ff60bc42bc', '_distance': 0.3309714198112488}),
  Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It

In [25]:
res['answer']

'Task decomposition is the process of breaking down a complex task into smaller, manageable subgoals or steps. It allows an agent to plan ahead and handle complex tasks more efficiently. Task decomposition can be done through various methods, such as using prompting techniques, task-specific instructions, or human inputs.'

In [27]:
for doc in res['context']:
    print(doc.page_content)

Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
T